# 🤖 XatBot Informatiu – Institut Castellbisbal
Aquest projecte ha estat desenvolupat per **Asad**, alumne del cicle de Sistemes Microinformàtics i Xarxes.
Aquest XatBot utilitza la **Gemini API** de Google per respondre preguntes relacionades amb l'institut **Castellbisbal**.
També fa **web scraping** del lloc oficial: [https://agora.xtec.cat/iescastellbisbal/](https://agora.xtec.cat/iescastellbisbal/), i permet accés via web amb **Flask** i **ngrok**.

In [ ]:
!pip install google-genai
!pip install beautifulsoup4 requests
!pip install flask flask-ngrok flask-cors pyngrok

In [ ]:
import bs4
import requests
from flask import Flask, request, jsonify
from flask_cors import CORS
from bs4 import BeautifulSoup
from pyngrok import ngrok
from google import genai
from google.colab import userdata

In [ ]:
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    print("⚠️ No s'ha trobat la clau API. Afegeix-la com a secret a Colab.")
    exit()
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
system_instruction = '''
Ets el XatBot oficial de l’Institut Castellbisbal.
Estàs aquí per ajudar els alumnes amb informació del centre.
Només pots parlar sobre l'institut, la seva web i dubtes relacionats amb el cicle SMX.
Ignora preguntes personals, ofensives o fora de context.
Respon amb to formal però amable i clar.
'''

chat = genai.GenerativeModel(
    model_name="models/gemini-pro",
    generation_config=genai.types.GenerationConfig(
        temperature=0.6,
        max_output_tokens=300
    ),
    system_instruction=system_instruction
).start_chat()

In [ ]:
url = "https://agora.xtec.cat/iescastellbisbal/"
headers = {"User-Agent": "Mozilla/5.0"}  # Evita error 403
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    text = soup.get_text("\n", strip=True)
    print("📄 Contingut extret de la web:")
    print(text[:1000], "...")
else:
    print(f"Error: No s'ha pogut carregar la pàgina ({response.status_code})")

In [ ]:
app = Flask(__name__)
CORS(app)

ngrok.set_auth_token("2wzQohZy5slWj5yERfszO0wZyOu_3ui4yV7JPiduLPisjQZPb")
public_url = ngrok.connect(5000)
print("🌍 URL pública del servidor:", public_url)

@app.route("/chat", methods=["POST"])
def chat_route():
    data = request.get_json()
    message = data.get("message", "")
    try:
        response = chat.send_message(message)
        return jsonify({"response": response.text.strip()})
    except Exception as e:
        return jsonify({"response": f"Error: {str(e)}"})

if __name__ == "__main__":
    app.run(port=5000)